## Fake News Classifier Using LSTM

In [1]:
import pandas as pd 


In [2]:
df=pd.read_csv("train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
### drop nan values
df=df.dropna()

In [5]:
# Get the Indepenent Feature
x = df.drop('label',axis=1)

In [6]:
# Get the dependent  value
y = df['label']

In [7]:
x.shape

(18285, 4)

In [8]:
y.shape

(18285,)

In [9]:
import tensorflow as tf


In [10]:
tf.__version__

'2.3.1'

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
## Vocabulary size
voc_size = 5000

## One Hot Representation

In [13]:
messages=x.copy()

In [15]:
messages.reset_index(inplace=True)

In [17]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\DELL
[nltk_data]     PC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

## Embedding Representation

In [25]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1291 1642  344]
 [   0    0    0 ... 2223 4746 3279]
 [   0    0    0 ... 2588 3942 3134]
 ...
 [   0    0    0 ... 4937 2048 2864]
 [   0    0    0 ...  805  904 1836]
 [   0    0    0 ... 1296 4383 3068]]


In [26]:
len(embedded_docs)

18285

In [42]:
from tensorflow.keras.layers import Dropout
### Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
len(embedded_docs),y.shape

(18285, (18285,))

In [44]:
import numpy as np 
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [45]:
x_final.shape,y_final.shape

((18285, 20), (18285,))

In [46]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=42)

## Model

In [47]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 31ms/step - loss: 0.3343 - accuracy: 0.8414 - val_loss: 0.1916 - val_accuracy: 0.9201
Epoch 2/10
192/192 [==============================] - 6s 31ms/step - loss: 0.1435 - accuracy: 0.9438 - val_loss: 0.1884 - val_accuracy: 0.9234
Epoch 3/10
192/192 [==============================] - 5s 28ms/step - loss: 0.1033 - accuracy: 0.9615 - val_loss: 0.2071 - val_accuracy: 0.9246
Epoch 4/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0809 - accuracy: 0.9692 - val_loss: 0.2139 - val_accuracy: 0.9203
Epoch 5/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0596 - accuracy: 0.9797 - val_loss: 0.2579 - val_accuracy: 0.9181
Epoch 6/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0426 - accuracy: 0.9858 - val_loss: 0.3188 - val_accuracy: 0.9183
Epoch 7/10
192/192 [==============================] - 5s 28ms/step - loss: 0.0324 - accuracy: 0.9895 - val_loss: 0.3392 - val_accuracy: 0.9158

## Performance Metrics and Accuracy

In [48]:
y_pred = model.predict_classes(x_test)

In [49]:
from sklearn.metrics import confusion_matrix

In [50]:
confusion_matrix(y_test,y_pred)

array([[3076,  343],
       [ 171, 2445]], dtype=int64)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9148301574150787